# Functional Closures and Decorators

##  Closures
* In order to understand closures, let's review the Python scoping rules: LEGB
  * L = local
  * E = enclosing
  * G = global
  * B = builtin (e.g., len() function)
  
        a = 'global scope'
   
        def outer_func():
            b = 'local to outer_func()'
            def inner_func():
                c = 'local to inner_func()'
                print(b, 'enclosing scope')
                print(a, 'global scope')
                
* When a function references a name that is not local, Python first attempts to resolve that name in the enclosing scope
* A *closure* is a nested function which remembers a value or values from the enclosing lexical scope even when the program flow is no longer in the enclosing scope

In [2]:
def make_adder(x):
    print('id(x): %X' % id(x))
    
    def adder(y):
        return x + y # Python uses LEGB to find 'x'
    
    print('id(adder): %X' % id(adder))
    return adder

add39 = make_adder(39)
add39(10)

id(x): 100273E60
id(adder): 105C23EA0


49

In [3]:
# let's use repr so we can see the address of the function
# we could use print("%X") as well...
repr(add39)

'<function make_adder.<locals>.adder at 0x105c23ea0>'

In [4]:
# all functions have a closure attribute
add39.__closure__

(<cell at 0x10561df18: int object at 0x100273e60>,)

In [5]:
# notice that the cell object has a reference to an int object
add39.__closure__[0].cell_contents

39

* One case where closures are frequently used is in building function wrappers
* Suppose we want to log each invocation of a function:

In [7]:
def logging(f):
    def wrapper(*args, **kwargs):
        print('Calling %r(%r, %r)' % (f, args, kwargs))
        return f(*args, **kwargs)
    return wrapper

logging_add39 = logging(add39)
print(add39(5))
logging_add39(10)

44
Calling <function make_adder.<locals>.adder at 0x105c23ea0>((10,), {})


49

In [10]:
logging_add39.__closure__[0].cell_contents

<function __main__.make_adder.<locals>.adder>

## Decorators
* Wrapper functions are so common, that Python has it's own term for it–a *decorator*.
* Why might you want to use a decorator?
  * sometimes you want to modify a function’s behavior without explicitly modifying the function, e.g., pre/post actions, debugging, etc. 
  * suppose we have a set of tasks that need to be performed by many different functions, e.g.,
   * access control
   * cleanup
   * error handling
   * logging
 * ...in other words, there is some boilerplate code that needs to be executed before or after  every invocation of the function


## Decorators build on topics we already know...
* nested functions
* variable positional args (`*args`)
* variable keyword args (`**kwargs`)
* functions are objects (actually everything in Python is an object)

In [2]:
def document_it(func):
    # below is a nested, or inner function
    def new_function(*args, **kwargs):
        print('Running function: {}'.format(func.__name__))
        print('Positional arguments: {}'.format(args))
        print('Keyword arguments: {}'.format(kwargs))
        # here we invoke the function passed in as an argument
        result = func(*args, **kwargs)
        print('Result: {}'.format(result))
        return result
    
    # document_it() is returning a reference to the inner function
    return new_function

def add_ints(a, b):
    return a + b

print('Running plain old add_ints()')
print(add_ints(3, 5))

# manual decorator assignment
cooler_add_ints = document_it(add_ints) 

print('Running cooler add_ints()')
cooler_add_ints(3, 5)

Running plain old add_ints()
8
Running cooler add_ints()
Running function: add_ints
Positional arguments: (3, 5)
Keyword arguments: {}
Result: 8


8

In [3]:
# decorator shorthand for what we did above
@document_it
def add_ints(a, b):
    return a + b

add_ints(4, 7)

Running function: add_ints
Positional arguments: (4, 7)
Keyword arguments: {}
Result: 11


11

## Lab: Decorators
1. Create a function called __`printer`__ that takes a string and prints it
  * Then create a wrapper that will print the number of times each letter appears in the string passed in to __`printer`__, followed by the string.
  * Use the wrapper as a decorator on your __`printer`__ function.
* Create a class, __`Coord2D`__, which represents a point in 2-D space (i.e., x and y coordinates). Implement the following methods:
 * __`distance()`__, which computes the distance between the current __`Coord2D`__ object and another __`Coord2D`__ object, i.e., __`d = c1.distance(c2)`__, where c1 and c2 are __`Coord2D`__ objects and d is a scalar–use __`math.sqrt`__ or __`math.hypot`__
 * __`add()`__: adds a __`Coord2D`__ object to the current one and returns a new __`Coord2D`__ object which represents their sum, i.e., __`c3 = c1.add(c2)`__
 * __`sub()`__, which subtracts a __`Coord2D`__ object from the current one, i.e. __`c3 = c1.sub(c2)`__
 * __`noneg()`__, which returns a new __`Coord2D`__ object whose coordinates are not negative, so __`c1 = c1.noneg()`__ would ensure __`c1`__ has nonnegative coordinates
* Use a decorator to ensure that the arguments to a function, as well as its return value, are nonnegative. Decorate the __`add()`__ and __`sub()`__ methods


In [ ]:
def counterwrap(func):
    def inner(s):
        from collections import Counter
        count = Counter(s)
        for item in cou

In [4]:
def counterwrap(func):
    def inner(s):
        from collections import Counter
        count = Counter(s)
        for item in count:
            print(item, '=>', count[item])
        func(s)
    
    return inner

def printer(s):
    print(s)

printer('Mississippi')

Mississippi


In [3]:
def counterwrap(func):
    def inner(s):
        from collections import Counter
        count = Counter(s)
        for item in count:
            print(item, count[item])
        print('    about to execute', func)
        func(s)

    return inner

@counterwrap
@document_it_now
def printer(s):
    print(s)

#twice_decorated_printer = counterwrap(document_it_now(printer))

print("Executing once_decorated_printer...")
#once_decorated_printer('hello')
printer('hello')
print()
#print("Executing twice_decorated_printer...")
#twice_decorated_printer('hello')

NameError: name 'document_it_now' is not defined

In [37]:
from math import sqrt

def ensure_positive(fn): # name of decorator
    def decorator(a, b): # actual decorator method
        a = a.noneg()
        b = b.noneg()
        print('type(a)=', type(a))
        print('type(b)=', type(b))
        return fn(a, b).noneg()
    return decorator

class Coord2d():
    def __init__(self, x=0, y=0):
        self.x, self.y = x, y

    def __repr__(self):
        return 'Coord2d(x={}, y={})'.format(self.x, self.y)

    def distance(self, other):
        x1, y1 = self.x, self.y
        x2, y2 = other.x, other.y
        return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

    @ensure_positive
    def add(self, other):
        return Coord2d(self.x + other.x, self.y + other.y)

    @ensure_positive
    def sub(self, other):
        return Coord2d(self.x - other.x, self.y - other.y)

    def noneg(self):
        return Coord2d(max(self.x, 0), max(self.y, 0))

In [38]:
c1 = Coord2d(1, 2)
c2 = Coord2d(-2, -3)

In [39]:
c1, c2

(Coord2d(x=1, y=2), Coord2d(x=-2, y=-3))

In [34]:
print(c1, c2)

Coord2d(x=1, y=2) Coord2d(x=-2, y=-3)


In [35]:
c1.distance(c2)

5.830951894845301

In [40]:
c1.add(c2)

type(a)= <class '__main__.Coord2d'>
type(b)= <class '__main__.Coord2d'>


Coord2d(x=1, y=2)